In [14]:
import pandas as pd

In [15]:
import numpy as np
from sklearn.metrics import confusion_matrix

def custom_classification_report(y_true, y_pred, target_names, class_indices):
    # Compute the confusion matrix
    cm = confusion_matrix(y_true, y_pred)

    # Initialize lists to store precision, recall, F1-score, and support for each class
    precision = []
    recall = []
    f1_score = []
    support = []

    # Calculate precision, recall, F1-score for each class
    for i in range(len(target_names)):
        true_positives = cm[i, i]
        false_positives = cm[:, i].sum() - true_positives
        false_negatives = cm[i, :].sum() - true_positives
        true_negatives = cm.sum() - (true_positives + false_positives + false_negatives)

        # Precision: TP / (TP + FP)
        if true_positives + false_positives > 0:
            precision_i = true_positives / (true_positives + false_positives)
        else:
            precision_i = 0.0

        # Recall: TP / (TP + FN)
        if true_positives + false_negatives > 0:
            recall_i = true_positives / (true_positives + false_negatives)
        else:
            recall_i = 0.0

        # F1-Score: 2 * (Precision * Recall) / (Precision + Recall)
        if precision_i + recall_i > 0:
            f1_i = 2 * (precision_i * recall_i) / (precision_i + recall_i)
        else:
            f1_i = 0.0

        # Support: The number of true instances of each class
        support_i = cm[i, :].sum()

        # Append calculated metrics for this class
        precision.append(precision_i)
        recall.append(recall_i)
        f1_score.append(f1_i)
        support.append(support_i)

    # Calculate accuracy
    accuracy = np.trace(cm) / np.sum(cm)

    # Calculate average F1-score for specified classes
    f1_average = np.mean([f1_score[i] for i in class_indices])

    # Print the aesthetically improved report
    print("\n" + "Classification Report".center(65, "="))
    print(f"{'Class':<15}{'Precision':>12}{'Recall':>12}{'F1-Score':>12}{'Support':>12}")
    print("=" * 65)
    for i, label in enumerate(target_names):
        print(f"{label:<15}{precision[i]:>12.4f}{recall[i]:>12.4f}{f1_score[i]:>12.4f}{support[i]:>12}")
    print("=" * 65)
    print(f"Average F1-Score for classes : {f1_average:.4f}")
    print(f"Accuracy : {accuracy:.4f}")
    print("=" * 65)

class_indices = [0, 1, 2]

In [16]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [17]:
params = {
    'objective': 'multi:softmax',
    'num_class': 3,  # Number of classes
    'learning_rate': 0.001,
    'max_depth': 10,
    'subsample': 0.6478457734751482,
    'colsample_bytree': 0.9431214021788126,
    'random_state': 42,
}

def xgBoostPrediction(X_train, y_train, X_test, y_test, params, num_boost_round=462, target_names=None):

    train_data = xgb.DMatrix(X_train, label=y_train)
    test_data = xgb.DMatrix(X_test, label=y_test)

    xgb_model = xgb.train(params, train_data,num_boost_round=num_boost_round)

    # Predict the class labels
    y_pred = xgb_model.predict(test_data)
    
    print('Expected features:', xgb_model.feature_names)
    print('Dataset columns:', X_train.columns.tolist())

    # Generate and print classification report
    print("xgBoost results:")
    custom_classification_report(y_test, y_pred, target_names=target_names,class_indices = [0, 1, 2])

In [18]:
from sklearn.ensemble import RandomForestClassifier

def rfPrediction(X_train, y_train, X_test, y_test, n_estimators=100, max_depth=None, target_names=None):
    # creating a RF classifier
    clf = RandomForestClassifier(n_estimators = n_estimators, max_depth=max_depth, random_state=42)  
    
    # Training the model on the training dataset
    # fit function is used to train the model using the training sets as parameters
    clf.fit(X_train, y_train)
    
    # performing predictions on the test dataset
    y_pred_rf = clf.predict(X_test)
    
    print("Random Forest results:")
    custom_classification_report(y_test, y_pred_rf, target_names=target_names,class_indices = [0, 1, 2])

In [19]:
from sklearn.linear_model import LogisticRegression

def logisticPrediction(X_train, y_train, X_test, y_test, max_iter=1000, target_names=None):
    # creating a RF classifier
    log_reg = LogisticRegression(random_state=42, max_iter=max_iter)
    
    # Training the model on the training dataset
    # fit function is used to train the model using the training sets as parameters
    log_reg.fit(X_train, y_train)
    
    # performing predictions on the test dataset
    y_pred_log_reg = log_reg.predict(X_test)
    
    print("Logistic Regression results:")
    custom_classification_report(y_test, y_pred_log_reg, target_names=target_names,class_indices = [0, 1, 2])

In [20]:
import torch
from torch.utils.data import DataLoader, TensorDataset
import os

import torch.nn as nn
import torch.optim as optim

# Define the LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, dropout=0.0):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(
            input_size, 
            hidden_size, 
            num_layers, 
            batch_first=True, 
            dropout=dropout if num_layers > 1 else 0
        )
        self.fc = nn.Linear(hidden_size, num_classes)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        # Ensure h0 and c0 are on the same device as input
        h0 = torch.zeros(self.lstm.num_layers, x.size(0), self.lstm.hidden_size).to(x.device)
        c0 = torch.zeros(self.lstm.num_layers, x.size(0), self.lstm.hidden_size).to(x.device)
        
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(self.dropout(out[:, -1, :]))
        return out

In [21]:
from sklearn.model_selection import ParameterGrid

def hyperparameter_tuning(X_train, y_train, X_val, y_val):
    """
    Perform hyperparameter tuning using ParameterGrid
    
    Args:
        X_train (torch.Tensor): Training input data
        y_train (torch.Tensor): Training labels
        X_val (torch.Tensor): Validation input data
        y_val (torch.Tensor): Validation labels
        column (list, optional): Feature column names
    
    Returns:
        dict: Best hyperparameters
    """
    # Convert to numpy if tensor
    X_train = X_train.numpy() if torch.is_tensor(X_train) else X_train
    y_train = y_train.numpy() if torch.is_tensor(y_train) else y_train
    X_val = X_val.numpy() if torch.is_tensor(X_val) else X_val
    y_val = y_val.numpy() if torch.is_tensor(y_val) else y_val

    # Convert to torch tensors
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.long)
    X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
    y_val_tensor = torch.tensor(y_val, dtype=torch.long)

    # Define hyperparameter search space
    param_grid = {
        'hidden_size': [32, 64],
        'num_layers': [2, 4],
        'learning_rate': [0.001],
        'batch_size': [1 ,2, 4],
        'dropout': [0.0, 0.2],
        'weight_decay': [0],
        'num_epochs': [20, 30, 50]
    }
    
    # Generate all combinations
    param_combinations = list(ParameterGrid(param_grid))
    
    # Store results
    results = []
    
    # Prepare input parameters
    input_size = X_train_tensor.shape[2]
    num_classes = len(torch.unique(y_train_tensor))
    
    # Search through hyperparameters
    for params in param_combinations:
        # Create model
        model = LSTMModel(
            input_size=input_size,
            hidden_size=params['hidden_size'],
            num_layers=params['num_layers'],
            num_classes=num_classes,
            dropout=params['dropout']
        )
        
        # Create DataLoaders
        train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
        val_dataset = TensorDataset(X_val_tensor, y_val_tensor)
        
        train_loader = DataLoader(train_dataset, batch_size=params['batch_size'], shuffle=False)
        val_loader = DataLoader(val_dataset, batch_size=params['batch_size'], shuffle=False)
        
        # Loss and Optimizer
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(
            model.parameters(), 
            lr=params['learning_rate'], 
            weight_decay=params['weight_decay']
        )
        
        # Training loop
        best_val_accuracy = 0
        for epoch in range(params['num_epochs']):
            model.train()
            for X_batch, y_batch in train_loader:
                optimizer.zero_grad()
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                loss.backward()
                optimizer.step()
            
            # Validation
            model.eval()
            correct = 0
            total = 0
            with torch.no_grad():
                for X_batch, y_batch in val_loader:
                    outputs = model(X_batch)
                    _, predicted = torch.max(outputs.data, 1)
                    total += y_batch.size(0)
                    correct += (predicted == y_batch).sum().item()
            
            val_accuracy = 100 * correct / total
            best_val_accuracy = max(best_val_accuracy, val_accuracy)
        
        # Store results
        results.append((params, best_val_accuracy))
        best_params, best_accuracy = max(results, key=lambda x: x[1])
        print(f"Current Best Hyperparameters: {best_params}")
        print(f"Current Best Validation Accuracy: {best_accuracy:.2f}%")
    
    # Find best hyperparameters
    best_params, best_accuracy = max(results, key=lambda x: x[1])
    
    print(f"Best Hyperparameters: {best_params}")
    print(f"Best Validation Accuracy: {best_accuracy:.2f}%")
    
    return best_params

In [22]:
if not os.path.exists('processed_data'):
    os.makedirs('processed_data')

list_of_data = []

    
for data in os.listdir('processed_data'):
    list_of_data.append(f"processed_data\\{data}")
    

In [23]:
def DataEvaluation(dataset):
    transform_eeg_data = pd.read_csv(dataset)
    le = LabelEncoder()
    transform_eeg_data['GROUP'] = le.fit_transform(transform_eeg_data['GROUP'])
    # Convert object columns to numeric
    for col in transform_eeg_data.columns:
        if transform_eeg_data[col].dtype == 'object':
            transform_eeg_data[col] = pd.to_numeric(transform_eeg_data[col], errors='coerce')

    # Features and target
    X = transform_eeg_data.drop('GROUP', axis=1)
    y = transform_eeg_data['GROUP']


    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)


    target_names=list(map(str,le.classes_))

    file_name = dataset.split('.')[0]
    print(f"For {file_name} data")

    # xgBoostPrediction(X_train, y_train, X_test, y_test, target_names=target_names, params=params)
    # rfPrediction(X_train, y_train, X_test,  y_test, target_names=target_names, max_depth=100)
    # logisticPrediction(X_train, y_train, X_test, y_test, target_names=target_names)



    # Prepare the data
    X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32).unsqueeze(1)
    y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
    X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32).unsqueeze(1)
    y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

    train_loader = DataLoader(train_dataset, batch_size=1, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)

    # {'batch_size': 1, 'dropout': 0.2, 'hidden_size': 64, 'learning_rate': 0.001, 'num_epochs': 50, 'num_layers': 4, 'weight_decay': 0}
    # Best Hyperparameters: {'batch_size': 4, 'dropout': 0.2, 'hidden_size': 32, 'learning_rate': 0.001, 'num_epochs': 30, 'num_layers': 2, 'weight_decay': 0}
    # Initialize the model, loss function, and optimizer
    input_size = X_train.shape[1]
    hidden_size = 64
    num_layers = 4
    num_classes = 3
    dropout = 0.2

    model = LSTMModel(input_size, hidden_size, num_layers, num_classes, dropout=dropout)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=0)

    # Train the model
    num_epochs = 50
    for epoch in range(num_epochs):
        model.train()
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
        
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
        

    # Evaluate the model
    model.eval()
    y_true = []
    y_pred = []
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            outputs = model(X_batch)
            _, predicted = torch.max(outputs.data, 1)
            y_true.extend(y_batch.numpy())
            y_pred.extend(predicted.numpy())


    print("LSTM results:")
    custom_classification_report(y_true, y_pred, target_names=target_names,class_indices = [0, 1, 2])



In [24]:
DataEvaluation(list_of_data[0])

For processed_data\Non_Standardized_data data
Epoch [1/50], Loss: 0.9328
Epoch [2/50], Loss: 0.8408
Epoch [3/50], Loss: 0.7188
Epoch [4/50], Loss: 0.5676
Epoch [5/50], Loss: 0.4745
Epoch [6/50], Loss: 0.5512
Epoch [7/50], Loss: 0.5691
Epoch [8/50], Loss: 0.5483
Epoch [9/50], Loss: 0.5314
Epoch [10/50], Loss: 0.5659
Epoch [11/50], Loss: 0.6115
Epoch [12/50], Loss: 0.6332
Epoch [13/50], Loss: 0.5998
Epoch [14/50], Loss: 0.5352
Epoch [15/50], Loss: 0.6872
Epoch [16/50], Loss: 0.6058
Epoch [17/50], Loss: 0.6161
Epoch [18/50], Loss: 0.6208
Epoch [19/50], Loss: 0.6251
Epoch [20/50], Loss: 0.6154
Epoch [21/50], Loss: 0.5884
Epoch [22/50], Loss: 0.6470
Epoch [23/50], Loss: 0.6541
Epoch [24/50], Loss: 0.6787
Epoch [25/50], Loss: 0.6973
Epoch [26/50], Loss: 0.6158
Epoch [27/50], Loss: 0.7472
Epoch [28/50], Loss: 0.6923
Epoch [29/50], Loss: 0.6994
Epoch [30/50], Loss: 0.6962
Epoch [31/50], Loss: 0.7075
Epoch [32/50], Loss: 0.7473
Epoch [33/50], Loss: 0.7129
Epoch [34/50], Loss: 0.6501
Epoch [35/5

In [25]:
DataEvaluation(list_of_data[1])

For processed_data\Pure_data data
Epoch [1/50], Loss: 1.0397
Epoch [2/50], Loss: 0.9507
Epoch [3/50], Loss: 0.6380
Epoch [4/50], Loss: 0.4953
Epoch [5/50], Loss: 0.5749
Epoch [6/50], Loss: 0.6319
Epoch [7/50], Loss: 0.5310
Epoch [8/50], Loss: 0.5979
Epoch [9/50], Loss: 0.5504
Epoch [10/50], Loss: 0.5648
Epoch [11/50], Loss: 0.5841
Epoch [12/50], Loss: 0.5992
Epoch [13/50], Loss: 0.5888
Epoch [14/50], Loss: 0.6136
Epoch [15/50], Loss: 0.5703
Epoch [16/50], Loss: 0.6200
Epoch [17/50], Loss: 0.6047
Epoch [18/50], Loss: 0.5395
Epoch [19/50], Loss: 0.5932
Epoch [20/50], Loss: 0.6539
Epoch [21/50], Loss: 0.6120
Epoch [22/50], Loss: 0.5811
Epoch [23/50], Loss: 0.5604
Epoch [24/50], Loss: 0.5964
Epoch [25/50], Loss: 0.6254
Epoch [26/50], Loss: 0.6039
Epoch [27/50], Loss: 0.5818
Epoch [28/50], Loss: 0.5998
Epoch [29/50], Loss: 0.5076
Epoch [30/50], Loss: 0.4987
Epoch [31/50], Loss: 0.5729
Epoch [32/50], Loss: 0.5970
Epoch [33/50], Loss: 0.6168
Epoch [34/50], Loss: 0.6270
Epoch [35/50], Loss: 0.

In [26]:
DataEvaluation(list_of_data[2])

For processed_data\Standardized_data data
Epoch [1/50], Loss: 1.0887
Epoch [2/50], Loss: 1.0223
Epoch [3/50], Loss: 0.7985
Epoch [4/50], Loss: 0.4032
Epoch [5/50], Loss: 0.2401
Epoch [6/50], Loss: 0.2474
Epoch [7/50], Loss: 0.2609
Epoch [8/50], Loss: 0.1858
Epoch [9/50], Loss: 0.2220
Epoch [10/50], Loss: 0.2111
Epoch [11/50], Loss: 0.0821
Epoch [12/50], Loss: 0.1488
Epoch [13/50], Loss: 0.1556
Epoch [14/50], Loss: 0.1375
Epoch [15/50], Loss: 0.0592
Epoch [16/50], Loss: 0.0350
Epoch [17/50], Loss: 0.0322
Epoch [18/50], Loss: 0.0300
Epoch [19/50], Loss: 0.0107
Epoch [20/50], Loss: 0.0200
Epoch [21/50], Loss: 0.0081
Epoch [22/50], Loss: 0.0128
Epoch [23/50], Loss: 0.0184
Epoch [24/50], Loss: 0.0039
Epoch [25/50], Loss: 0.0130
Epoch [26/50], Loss: 0.0042
Epoch [27/50], Loss: 0.0031
Epoch [28/50], Loss: 0.0102
Epoch [29/50], Loss: 0.0028
Epoch [30/50], Loss: 0.0064
Epoch [31/50], Loss: 0.0092
Epoch [32/50], Loss: 0.0012
Epoch [33/50], Loss: 0.0079
Epoch [34/50], Loss: 0.0107
Epoch [35/50], 

In [29]:
DataEvaluation(list_of_data[3])

For processed_data\Standardized_pure_data data
Epoch [1/50], Loss: 1.0590
Epoch [2/50], Loss: 0.9391
Epoch [3/50], Loss: 0.6203
Epoch [4/50], Loss: 0.2913
Epoch [5/50], Loss: 0.1240
Epoch [6/50], Loss: 0.0904
Epoch [7/50], Loss: 0.0377
Epoch [8/50], Loss: 0.0163
Epoch [9/50], Loss: 0.0061
Epoch [10/50], Loss: 0.0047
Epoch [11/50], Loss: 0.0023
Epoch [12/50], Loss: 0.0021
Epoch [13/50], Loss: 0.0107
Epoch [14/50], Loss: 0.0017
Epoch [15/50], Loss: 0.0038
Epoch [16/50], Loss: 0.0006
Epoch [17/50], Loss: 0.0002
Epoch [18/50], Loss: 0.0010
Epoch [19/50], Loss: 0.0010
Epoch [20/50], Loss: 0.0002
Epoch [21/50], Loss: 0.0014
Epoch [22/50], Loss: 0.0012
Epoch [23/50], Loss: 0.0046
Epoch [24/50], Loss: 0.0005
Epoch [25/50], Loss: 0.0005
Epoch [26/50], Loss: 0.0001
Epoch [27/50], Loss: 0.0003
Epoch [28/50], Loss: 0.0005
Epoch [29/50], Loss: 0.0003
Epoch [30/50], Loss: 0.0001
Epoch [31/50], Loss: 0.0001
Epoch [32/50], Loss: 0.0001
Epoch [33/50], Loss: 0.0001
Epoch [34/50], Loss: 0.0004
Epoch [35/

In [28]:
# LSTM Hyperparameter Tuning
def LSTM_Hyper_tuning(dataset):
    transform_eeg_data = pd.read_csv(dataset)
    le = LabelEncoder()
    transform_eeg_data['GROUP'] = le.fit_transform(transform_eeg_data['GROUP'])
    # Convert object columns to numeric
    for col in transform_eeg_data.columns:
        if transform_eeg_data[col].dtype == 'object':
            transform_eeg_data[col] = pd.to_numeric(transform_eeg_data[col], errors='coerce')

    # Features and target
    X = transform_eeg_data.drop('GROUP', axis=1)
    y = transform_eeg_data['GROUP']


    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)


    target_names=list(map(str,le.classes_))

    file_name = dataset.split('.')[0]
    print(f"For {file_name} data")

    # Prepare the data
    X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32).unsqueeze(1)
    y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)
    X_test_tensor = torch.tensor(X_test.values, dtype=torch.float32).unsqueeze(1)
    y_test_tensor = torch.tensor(y_test.values, dtype=torch.long)

    print("\n Hyperparameter Tuning on LSTM:")
    hyperparameter_tuning(X_train=X_train_tensor, y_train=y_train_tensor, X_val=X_test_tensor, y_val=y_test_tensor)

In [ ]:
from sklearn.model_selection import GridSearchCV

def xg_Hyper_tuning(dataset):

    transform_eeg_data = pd.read_csv(dataset)
    le = LabelEncoder()
    transform_eeg_data['GROUP'] = le.fit_transform(transform_eeg_data['GROUP'])
    # Convert object columns to numeric
    for col in transform_eeg_data.columns:
        if transform_eeg_data[col].dtype == 'object':
            transform_eeg_data[col] = pd.to_numeric(transform_eeg_data[col], errors='coerce')

    # Features and target
    X = transform_eeg_data.drop('GROUP', axis=1)
    y = transform_eeg_data['GROUP']


    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)


    target_names=list(map(str,le.classes_))

    file_name = dataset.split('.')[0]
    print(f"For {file_name} data")

    param_grid = {
        'objective': ['multi:softmax'],
        'n_estimators': [50, 100, 200],
        'learning_rate': [0.001 ,0.01, 0.1, 0.2],
        'max_depth': [3, 5, 7, 10],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'random_state': [42],
        'num_class': [3]
    }

    grid_search = GridSearchCV(estimator=xgb, param_grid=param_grid, 
                           scoring='accuracy', cv=3, verbose=2, n_jobs=-1)

# Perform grid search
    grid_search.fit(X_train, y_train)
        # Best parameters and score
    print("Best Parameters:", grid_search.best_params_)
    print("Best Score:", grid_search.best_score_)

    # Evaluate the model on the test set
    best_model = grid_search.best_estimator_
    test_accuracy = best_model.score(X_test, y_test)
    print("Test Accuracy:", test_accuracy)


In [31]:
LSTM_Hyper_tuning(list_of_data[2])

For processed_data\Standardized_data data

 Hyperparameter Tuning on LSTM:
Current Best Hyperparameters: {'batch_size': 1, 'dropout': 0.0, 'hidden_size': 32, 'learning_rate': 0.001, 'num_epochs': 20, 'num_layers': 2, 'weight_decay': 0}
Current Best Validation Accuracy: 65.00%
Current Best Hyperparameters: {'batch_size': 1, 'dropout': 0.0, 'hidden_size': 32, 'learning_rate': 0.001, 'num_epochs': 20, 'num_layers': 4, 'weight_decay': 0}
Current Best Validation Accuracy: 70.00%
Current Best Hyperparameters: {'batch_size': 1, 'dropout': 0.0, 'hidden_size': 32, 'learning_rate': 0.001, 'num_epochs': 20, 'num_layers': 4, 'weight_decay': 0}
Current Best Validation Accuracy: 70.00%
Current Best Hyperparameters: {'batch_size': 1, 'dropout': 0.0, 'hidden_size': 32, 'learning_rate': 0.001, 'num_epochs': 20, 'num_layers': 4, 'weight_decay': 0}
Current Best Validation Accuracy: 70.00%
Current Best Hyperparameters: {'batch_size': 1, 'dropout': 0.0, 'hidden_size': 32, 'learning_rate': 0.001, 'num_epoch

KeyboardInterrupt: 

In [ ]:
LSTM_Hyper_tuning(list_of_data[1])

In [30]:
LSTM_Hyper_tuning(list_of_data[3])

For processed_data\Standardized_pure_data data

 Hyperparameter Tuning on LSTM:
Current Best Hyperparameters: {'batch_size': 1, 'dropout': 0.0, 'hidden_size': 32, 'learning_rate': 0.001, 'num_epochs': 20, 'num_layers': 2, 'weight_decay': 0}
Current Best Validation Accuracy: 45.00%
Current Best Hyperparameters: {'batch_size': 1, 'dropout': 0.0, 'hidden_size': 32, 'learning_rate': 0.001, 'num_epochs': 20, 'num_layers': 2, 'weight_decay': 0}
Current Best Validation Accuracy: 45.00%
Current Best Hyperparameters: {'batch_size': 1, 'dropout': 0.0, 'hidden_size': 32, 'learning_rate': 0.001, 'num_epochs': 20, 'num_layers': 2, 'weight_decay': 0}
Current Best Validation Accuracy: 45.00%
Current Best Hyperparameters: {'batch_size': 1, 'dropout': 0.0, 'hidden_size': 32, 'learning_rate': 0.001, 'num_epochs': 30, 'num_layers': 4, 'weight_decay': 0}
Current Best Validation Accuracy: 50.00%
Current Best Hyperparameters: {'batch_size': 1, 'dropout': 0.0, 'hidden_size': 32, 'learning_rate': 0.001, 'num_

KeyboardInterrupt: 

#### First LSTM Hyperparameter Tuning on Standardized Data:
***
Param:
    param_grid = {
        'hidden_size': [64, 128],
        'num_layers': [1, 2],
        'learning_rate': [0.001],
        'batch_size': [4],
        'dropout': [0.0, 0.2, 0.5],
        'weight_decay': [0, 1e-5],
        'num_epochs': [50, 70]
    }
-    Train_Shuffle = False
-    Test_Shuffle = False

- Best Hyperparameters: {'batch_size': 4, 'dropout': 0.0, 'hidden_size': 64, 'learning_rate': 0.001, 'num_epochs': 50, 'num_layers': 2, 'weight_decay': 0}
- Best Validation Accuracy: 75.00%
- Best Average F1-Score for classes : 0.6868
***


#### Second LSTM Hyperparameter Tuning on Standardized Data:
***
Param:
    param_grid = {
        'hidden_size': [64, 128, 256],
        'num_layers': [2, 4, 8, 16],
        'learning_rate': [0.001],
        'batch_size': [4, 8, 16],
        'dropout': [0.0, 0.2],
        'weight_decay': [0, 1e-4],
        'num_epochs': [50]
    }
-    Train_Shuffle = False
-    Test_Shuffle = False

- Best Hyperparameters: {'batch_size': 4, 'dropout': 0.0, 'hidden_size': 64, 'learning_rate': 0.001, 'num_epochs': 50, 'num_layers': 4, 'weight_decay': 0}
- Best Validation Accuracy: 80.00%
- Best Average F1-Score for classes : 0.8004
***

#### Third LSTM Hyperparameter Tuning on Standardized Data:
***
Param:
    param_grid = {
        'hidden_size': [64, 128],
        'num_layers': [2, 4],
        'learning_rate': [0.001],
        'batch_size': [1 ,2, 4],
        'dropout': [0.0, 0.2],
        'weight_decay': [0, 1e-4],
        'num_epochs': [20, 30, 50]
    }
-    Train_Shuffle = False
-    Test_Shuffle = False

- Best Hyperparameters: {'batch_size': 1, 'dropout': 0.0, 'hidden_size': 64, 'learning_rate': 0.001, 'num_epochs': 20, 'num_layers': 4, 'weight_decay': 0}
- Best Validation Accuracy: 80.00%
- Best Average F1-Score for classes : 0.8426
***

#### Fourth LSTM Hyperparameter Tuning on Standardized Data:
***
Param:
    param_grid = {
        'hidden_size': [32, 64, 128],
        'num_layers': [2, 4],
        'learning_rate': [0.001],
        'batch_size': [1 ,2, 4],
        'dropout': [0.0, 0.2],
        'weight_decay': [0, 1e-4],
        'num_epochs': [20, 30, 50]
    }
-    Train_Shuffle = False
-    Test_Shuffle = False

- Best Hyperparameters: {'batch_size': 1, 'dropout': 0.2, 'hidden_size': 64, 'learning_rate': 0.001, 'num_epochs': 50, 'num_layers': 4, 'weight_decay': 0}
- Best Validation Accuracy: 85.00%
- Best Average F1-Score for classes : ?? (Best Possible with 0.8004 along with 80.00% accuracy)
***

First LSTM Parameter Tuning on Non-Standardized Data:
***
Param:
    param_grid = {
        'hidden_size': [32, 64],
        'num_layers': [2, 4],
        'learning_rate': [0.001],
        'batch_size': [1 ,2, 4],
        'dropout': [0.0, 0.2],
        'weight_decay': [0],
        'num_epochs': [20, 30, 50]
    }
-    Train_Shuffle = False
-    Test_Shuffle = False

- Best Hyperparameters: {'batch_size': 4, 'dropout': 0.2, 'hidden_size': 32, 'learning_rate': 0.001, 'num_epochs': 30, 'num_layers': 2, 'weight_decay': 0}
- Best Validation Accuracy: 55.00%
- Best Average F1-Score for classes : 0.6868
***

In [585]:
# # Applying PCA function on training
# # and testing set of X component
# from sklearn.decomposition import PCA

# pca = PCA(n_components=0.90)

# X_train_pca = pca.fit_transform(X_train)
# X_test_pca = pca.transform(X_test)

# explained_variance = pca.explained_variance_ratio_

In [586]:
# explained_variance

In [587]:
# train_data_pca = xgb.DMatrix(X_train_pca, label=y_train)
# test_data_pca = xgb.DMatrix(X_test_pca, label=y_test)

# xgb_model_pca = xgb.train(params, train_data_pca,num_boost_round=462)

# # Predict the class labels
# y_pred_pca = xgb_model_pca.predict(test_data_pca)

In [588]:
# # Generate and print classification report
# report = custom_classification_report(y_test, y_pred_pca, target_names=target_names,class_indices = [0, 1, 2])

In [589]:
# from sklearn.ensemble import RandomForestClassifier


# # creating a RF classifier
# clf = RandomForestClassifier(n_estimators = 100)  
 
# # Training the model on the training dataset
# # fit function is used to train the model using the training sets as parameters
# clf.fit(X_train_pca, y_train)
 
# # performing predictions on the test dataset
# y_pred_rf = clf.predict(X_test_pca)
 
# custom_classification_report(y_test, y_pred_rf, target_names=target_names,class_indices = [0, 1, 2])

In [590]:
# from sklearn.linear_model import LogisticRegression

# # creating a RF classifier
# log_reg = LogisticRegression(random_state=42)
 
# # Training the model on the training dataset
# # fit function is used to train the model using the training sets as parameters
# log_reg.fit(X_train_pca, y_train)
 
# # performing predictions on the test dataset
# y_pred_log_reg = log_reg.predict(X_test_pca)
 
# custom_classification_report(y_test, y_pred_log_reg, target_names=target_names,class_indices = [0, 1, 2])

In [591]:
# import torch
# from torch.utils.data import DataLoader, TensorDataset
# from sklearn.metrics import f1_score, accuracy_score

# import torch.nn as nn
# import torch.optim as optim

# # Define the LSTM model
# class LSTMModel(nn.Module):
#     def __init__(self, input_size, hidden_size, num_layers, num_classes):
#         super(LSTMModel, self).__init__()
#         self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
#         self.fc = nn.Linear(hidden_size, num_classes)
    
#     def forward(self, x):
#         h0 = torch.zeros(num_layers, x.size(0), hidden_size).to(x.device)
#         c0 = torch.zeros(num_layers, x.size(0), hidden_size).to(x.device)
        
#         out, _ = self.lstm(x, (h0, c0))
#         out = self.fc(out[:, -1, :])  # Extract the output from the last time step
#         return out

# # Prepare the data
# X_train_pca_tensor = torch.tensor(X_train_pca, dtype=torch.float32).unsqueeze(1)
# y_train_pca_tensor = torch.tensor(y_train.values, dtype=torch.long)
# X_test_pca_tensor = torch.tensor(X_test_pca, dtype=torch.float32).unsqueeze(1)
# y_test_pca_tensor = torch.tensor(y_test.values, dtype=torch.long)

# train_pca_dataset = TensorDataset(X_train_pca_tensor, y_train_pca_tensor)
# test_pca_dataset = TensorDataset(X_test_pca_tensor, y_test_pca_tensor)

# train_pca_loader = DataLoader(train_pca_dataset, batch_size=4, shuffle=True)
# test_pca_loader = DataLoader(test_pca_dataset, batch_size=4, shuffle=True)

# # Initialize the model, loss function, and optimizer
# input_size = X_train.shape[1]
# hidden_size = 128
# num_layers = 4
# num_classes = 3

# model = LSTMModel(input_size, hidden_size, num_layers, num_classes)
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)

# # Train the model
# num_epochs = 50
# for epoch in range(num_epochs):
#     model.train()
#     for X_batch, y_batch in train_loader:
#         optimizer.zero_grad()
#         outputs = model(X_batch)
#         loss = criterion(outputs, y_batch)
#         loss.backward()
#         optimizer.step()
    
#     print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# # Evaluate the model
# model.eval()
# y_true = []
# y_pred = []
# with torch.no_grad():
#     for X_batch, y_batch in test_loader:
#         outputs = model(X_batch)
#         _, predicted = torch.max(outputs.data, 1)
#         y_true.extend(y_batch.numpy())
#         y_pred.extend(predicted.numpy())

# # Calculate F1 score and accuracy
# f1 = f1_score(y_true, y_pred, average='weighted')
# accuracy = accuracy_score(y_true, y_pred)

# print(f'F1 Score: {f1:.4f}')
# print(f'Accuracy: {accuracy * 100:.2f}%')